# Multi-Threading Performance Benchmark

This notebook contains a performance comparison of different methods to process the NDVI calculations.

The `%%timeit` cell magic runs the cell content multiple times and outputs statistics on those multiple runs, thereby reducing factors such as garbage collection pauses etc.

In [1]:
from multiprocessing import Pool, cpu_count
from numpy import ma
from pathlib import Path
import rasterio as r

In [2]:
test_files = list(Path('output/ndvi').glob('*.tif'))
print(f'Number of files: {len(test_files)}')

Number of files: 27


The function we test with:

In [3]:
def average(file_path):
    with r.open(file_path) as src:
        data = src.read(1, masked=True)
        return file_path, ma.average(data)

## In a single process
### Time to process a single file

In [4]:
%%timeit
average(test_files[0])

36.2 ms ± 42.6 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


### Time to process all files

In [5]:
%%timeit
averages = [avg for avg in map(average, test_files)]

980 ms ± 7.38 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### Increasing the list size

In [6]:
%%timeit
averages = [avg for avg in map(average, test_files * 5)]

4.86 s ± 10.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Time when using a worker pool

Number of CPUs the multiprocessing pools can access:

In [7]:
cpu_count()

4

### On One element

In [8]:
%%timeit
with Pool() as pool:
    averages = [avg for avg in pool.map(average, test_files[:1])]

277 ms ± 3.92 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### On the complete list

In [9]:
%%timeit
with Pool() as pool:
    averages = [avg for avg in pool.map(average, test_files)]

630 ms ± 8.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### Increasing the list size

In [10]:
%%timeit
with Pool() as pool:
    averages = [avg for avg in pool.map(average, test_files * 5)]

2.1 s ± 20 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Result

As we can see when processing a single element, multiprocessing comes with an overhead.
When the list to be processed is sufficiently large, we get a reduction in processing time of roughly 30%-50%, depending on list size.

Averaging the masked array is a fairly simple operation that scales in $O(N)$ with the size of the input array.
The time reduction should be even higher for more complex tasks.